In [40]:
import numpy as np
import pandas as pd


## SF Residents Analysis

In [174]:
df1 = pd.read_excel('dataset_sf.xlsx',sheet_name='Data')

In [175]:
df1.head()

,*RESPNUM,Q2,Trips,DA,DO,CRSHRE,TNC,TAXI,TRANS,SHTTLE,...,SCOOT,OTH-M-SF,DK-M-SF,Rideshare_use_freq,Rideshare_use_freq2,Taxi_use_freq,Car_ownership,Age,Income,Gender
0,3,9,4,2,2,0,0,0,0,0,...,0,0,0,4,1,1,1,4,6,2
1,9,9,9,0,2,0,0,0,6,0,...,0,0,0,5,1,5,2,3,1,2
2,82,9,7,2,0,0,0,0,4,0,...,0,0,0,5,1,6,1,3,1,2
3,256,9,2,0,2,0,0,0,0,0,...,0,0,0,1,1,1,2,6,1,2
4,673,9,4,0,0,0,0,0,4,0,...,0,0,0,1,1,1,2,5,1,2


In [176]:
def reAssignCode(x):
    if (x==1):
        return 7
    else: return (x)
#Frequency of taxi use code - in ascending order; 1 means never tried - reassigning that to 7
df1.Rideshare_use_freq = df1.Rideshare_use_freq.apply(reAssignCode)
df1.Rideshare_use_freq2 = df1.Rideshare_use_freq2.apply(reAssignCode)
df1.Taxi_use_freq = df1.Taxi_use_freq.apply(reAssignCode)


In [177]:
def combineFreq(x):
    a = min(x['Rideshare_use_freq'],x['Rideshare_use_freq2'],x['Taxi_use_freq'])
    return a
df1['overall_cab_freq'] = df1.apply(combineFreq,axis=1)
#Usage of specifically uber/lyft/chariot/traditional taxis does not matter, thus taking max frequency of either of these modes

In [178]:
def replaceFreq(x):
    if (x == 7 or x==6): 
        return "Never tried/Do not use"
    elif (x == 2 or x==3):
        return "Daily/Weekly"
    elif (x == 4 or x==5):
        return "Monthly/Rarely"

def replaceOwnership(x):
    if x==1:
        return "Yes"
    elif x==2:
        return "No"
    
def replaceAge(x):
    if x==1:
        return "18-24"
    elif x==2:
        return "25-34"
    elif x==3:
        return "35-44"
    elif x==4:
        return "45-54"
    elif x==5:
        return "55-64"
    elif x==6:
        return "65+"
    elif x==7: return "Refused"

def replaceIncome(x):
    if x==1:
        return "less than 15k"
    elif x==2:
        return "15k-25k"
    elif x==3:
        return "25k-35k"
    elif x==4:
        return "35k-75k"
    elif x==6:
        return "100k-200k"
    elif x==7:
        return "more than 200k"
    elif x==8: return "Refused"

In [179]:
df1.overall_cab_freq = df1.overall_cab_freq.apply(replaceFreq)
df1.Car_ownership = df1.Car_ownership.apply(replaceOwnership)
df1.Age = df1.Age.apply(replaceAge)
df1.Income = df1.Income.apply(replaceIncome)

In [181]:
groupObjt = df1.groupby(df1.Car_ownership)
carOwnershipTrips = groupObjt['Trips'].sum()
carOwnershipCount = groupObjt['Trips'].count()
MedianTripCount = groupObjt['Trips'].median()

In [182]:
carOwnershipTrips

Car_ownership
No      561
Yes    1793
Name: Trips, dtype: int64

In [183]:
carOwnershipCount

Car_ownership
No     102
Yes    299
Name: Trips, dtype: int64

In [184]:
MedianTripCount

Car_ownership
No     5
Yes    5
Name: Trips, dtype: int64

### Non Car Owners form the Majority of Potential Ride Share Users
* 25% of SF population does not own cars
* 24% of all SF trips are undertaken by non car owners
* Just because someone does not own a car does not mean they take lesser trips.
* Median monthly trip count is same for both car and non car users

In [185]:
transportBreakdown = df1[['Trips','DA','DO','CRSHRE','TNC','TAXI','TRANS','SHTTLE','BIKE','WALK','SCOOT','OTH-M-SF','DK-M-SF']].sum().to_frame(name='Count')

In [186]:
totalTrips = 2354
transportBreakdown['Percent']=(transportBreakdown.Count/totalTrips*100).astype('int')

In [187]:
transportBreakdown

,Count,Percent
Trips,2354,100
DA,630,26
DO,344,14
CRSHRE,4,0
TNC,90,3
TAXI,10,0
TRANS,544,23
SHTTLE,17,0
BIKE,49,2
WALK,633,26


* Above table shows % breakdown of mode of transport of all trips undertaken by SF residents
* Combining Different Modes to Get a better idea

In [188]:
df1['Personal_car'] = df1.DA + df1.DO + df1.CRSHRE
df1['Taxi_uber_lyft'] = df1.TNC + df1.TAXI
df1['Public_transport'] = df1.TRANS
df1['Shuttle']=df1.SHTTLE
df1['Bike_walk_scoot']= df1.BIKE + df1.WALK + df1.SCOOT
df1['Other_noresponse'] = df1['OTH-M-SF'] + df1['DK-M-SF']

In [189]:
transportBreakdown2 = df1[['Personal_car','Taxi_uber_lyft','Public_transport','Shuttle','Bike_walk_scoot','Other_noresponse']].sum().to_frame(name='Count')

In [190]:
transportBreakdown2['Percent']=(transportBreakdown2.Count/totalTrips*100).astype('int')

In [191]:
transportBreakdown2

,Count,Percent
Personal_car,978,41
Taxi_uber_lyft,100,4
Public_transport,544,23
Shuttle,17,0
Bike_walk_scoot,689,29
Other_noresponse,6,0


* 4% of all trips in SF are on ubers or cabs
* 23% of all trips in SF are on public transport
* All SF residents using taxi/uber and public transport would be potential Cruise Ride sharing users
* 27% of all trips in SF - fall under the total adressable market size for Cruise

### Cab Usage Frequency Breakdown for Population 

In [192]:
freqGrp = df1.groupby(df1.overall_cab_freq)
freq = freqGrp['Trips'].count().to_frame(name='Count')
totalRespondents = 401
freq['Percent']=(freq.Count/401*100).astype('int')
freq

,Count,Percent
overall_cab_freq,,
Daily/Weekly,97,24
Monthly/Rarely,220,54
Never tried/Do not use,84,20


* 24% of population uses some sort of cab on a daily/weekly basis. 54% of the population uses cabs on a monthly/rarely basis. This goes to show that a significant population of SF uses cab on a regular basis

### Identifying Cohorts to Roll out Service

In [193]:
popGrp = df1.groupby(['Age','Income'])
popGrp['Trips'].median()
# Shows avergae trip count in a month broken down by age, income 

Age      Income        
18-24    100k-200k         3.0
         15k-25k           6.0
         25k-35k           4.0
         35k-75k           3.0
         Refused           6.0
         less than 15k     5.0
25-34    100k-200k         5.0
         15k-25k           5.0
         25k-35k           8.0
         35k-75k           6.5
         Refused           4.0
         less than 15k     5.5
         more than 200k    6.0
35-44    100k-200k         5.5
         15k-25k           6.0
         25k-35k           7.5
         35k-75k           5.0
         Refused           7.0
         less than 15k     8.0
         more than 200k    7.0
45-54    100k-200k         6.0
         15k-25k           4.5
         25k-35k           6.0
         35k-75k           4.0
         Refused           5.0
         less than 15k     2.0
         more than 200k    5.0
55-64    100k-200k         5.0
         15k-25k           6.5
         25k-35k           4.0
         35k-75k           2.0
         Refuse

* Target cohorts with average trip count of 7 or more

## Outside SF Analysis

In [210]:
df2 = pd.read_excel('dataset_outside_sf.xlsx',sheet_name='Data')

In [211]:
# Calculatng how outside sf residents commuted to sf
def car(x):
    if x['Q7M'] in [1,2,3]:
        a = x['Q12']
    if x['Q14M'] in [1,2,3]:
        b = x['Q19']
    if x['Q7M'] not in [1,2,3]:
        a = 0
    if x['Q14M'] not in [1,2,3]:
        b = 0
    return (a+b)

def cab(x):
    if x['Q7M'] in [4,5]:
        a = x['Q12']
    if x['Q14M'] in [4,5]:
        b = x['Q19']
    if x['Q7M'] not in [4,5]:
        a = 0
    if x['Q14M'] not in [4,5]:
        b = 0
    return (a+b)

def public(x):
    if x['Q7M'] in [6]:
        a = x['Q12']
    if x['Q14M'] in [6]:
        b = x['Q19']
    if x['Q7M'] not in [6]:
        a = 0
    if x['Q14M'] not in [6]:
        b = 0
    return (a+b)

def shuttle(x):
    if x['Q7M'] in [7]:
        a = x['Q12']
    if x['Q14M'] in [7]:
        b = x['Q19']
    if x['Q7M'] not in [7]:
        a = 0
    if x['Q14M'] not in [7]:
        b = 0
    return (a+b)

def bike_walk(x):
    if x['Q7M'] in [8,9,10]:
        a = x['Q12']
    if x['Q14M'] in [8,9,10]:
        b = x['Q19']
    if x['Q7M'] not in [8,9,10]:
        a = 0
    if x['Q14M'] not in [8,9,10]:
        b = 0
    return (a+b)
def other(x):
    if x['Q7M'] in [11,12]:
        a = x['Q12']
    if x['Q14M'] in [11,12]:
        b = x['Q19']
    if x['Q7M'] not in [11,12]:
        a = 0
    if x['Q14M'] not in [11,12]:
        b = 0
    return (a+b)

                   

In [212]:
df2.Q12.fillna(0,inplace=True)
df2.Q19.fillna(0,inplace=True)

In [213]:
df2['personal_car']=df2.apply(car,axis=1)
df2['taxi_uber_lyft']=df2.apply(cab,axis=1)
df2['public']=df2.apply(public,axis=1)
df2['shuttle']=df2.apply(shuttle,axis=1)
df2['bike_walk_scoot']=df2.apply(bike_walk,axis=1)
df2['other_noresponse']=df2.apply(other,axis=1)
df2['total_sf_trips']= df2.Q12+df2.Q19

In [214]:
df2.head()

,*RESPNUM,Q2,Q3,Trips,Q6,Q10,Q11Total,Q12,Q19,Q13,...,Age,Income,Gender,personal_car,taxi_uber_lyft,public,shuttle,bike_walk_scoot,other_noresponse,total_sf_trips
0,2,1,1,19,5,1,3.0,3,1.0,1,...,3,1,1,3.0,0.0,1.0,0.0,0.0,0,4.0
1,38,7,1,33,10,1,2.0,8,1.0,1,...,5,5,1,9.0,0.0,0.0,0.0,0.0,0,9.0
2,105,5,1,125,20,1,2.0,25,5.0,1,...,3,4,2,5.0,0.0,25.0,0.0,0.0,0,30.0
3,111,7,1,48,21,1,2.0,9,12.0,1,...,4,4,1,9.0,0.0,0.0,0.0,12.0,0,21.0
4,136,2,1,6,2,2,NaN,1,1.0,1,...,5,4,2,0.0,0.0,2.0,0.0,0.0,0,2.0


In [215]:
transportBreakdown = df2[['personal_car','taxi_uber_lyft','public','shuttle','bike_walk_scoot','other_noresponse']].sum().to_frame(name='Count')

In [216]:
totalSFTrips = 3102

In [217]:
transportBreakdown['Percent']=(transportBreakdown.Count/totalSFTrips*100)

In [218]:
transportBreakdown

,Count,Percent
personal_car,1739.0,56.060606
taxi_uber_lyft,23.0,0.741457
public,1262.0,40.683430
shuttle,58.0,1.869761
bike_walk_scoot,20.0,0.644745
other_noresponse,0.0,0.000000


* 97% of outside SF population uses personal car or public transport
* Cruise should only start ride sharing from outside sf to sf, if prices are competitive with public transport
* for outside SF, ignoring their frequency of usage of taxi/lyft/uber as we are only concerned with their mode of transport from outside sf to sf - not their overall preferences

#### Breaking Down population that uses public transport into cohorts to see the group that travels the most

In [219]:
df2.Car_ownership = df2.Car_ownership.apply(replaceOwnership)
df2.Age = df2.Age.apply(replaceAge)
df2.Income = df2.Income.apply(replaceIncome)

In [220]:
def county(x):
    if x==1: return "Alameda"
    elif x==2: return "Contra Costa"
    elif x==3: return "Marin"
    elif x==4: return "Napa"
    elif x==5: return "San Mateo"
    elif x==6: return "Santa Clara"
    elif x==7: return "Solano"
    elif x==8: return "Sonoma"
    

In [221]:
df2.Q2 = df2.Q2.apply(county)

In [222]:
df2.head()

,*RESPNUM,Q2,Q3,Trips,Q6,Q10,Q11Total,Q12,Q19,Q13,...,Age,Income,Gender,personal_car,taxi_uber_lyft,public,shuttle,bike_walk_scoot,other_noresponse,total_sf_trips
0,2,Alameda,1,19,5,1,3.0,3,1.0,1,...,35-44,less than 15k,1,3.0,0.0,1.0,0.0,0.0,0,4.0
1,38,Solano,1,33,10,1,2.0,8,1.0,1,...,55-64,None,1,9.0,0.0,0.0,0.0,0.0,0,9.0
2,105,San Mateo,1,125,20,1,2.0,25,5.0,1,...,35-44,35k-75k,2,5.0,0.0,25.0,0.0,0.0,0,30.0
3,111,Solano,1,48,21,1,2.0,9,12.0,1,...,45-54,35k-75k,1,9.0,0.0,0.0,0.0,12.0,0,21.0
4,136,Contra Costa,1,6,2,2,NaN,1,1.0,1,...,55-64,35k-75k,2,0.0,0.0,2.0,0.0,0.0,0,2.0


In [232]:
cohorts = df2[df2.public!=0].groupby(['Q2','Age'])

In [233]:
cohorts['total_sf_trips'].median()

Q2            Age    
Alameda       18-24       1.5
              25-34      14.0
              35-44       3.0
              45-54       2.5
              55-64       2.0
              65+         2.5
              Refused    20.0
Contra Costa  18-24       4.0
              25-34       5.0
              35-44       7.0
              45-54       5.0
              55-64       3.0
              65+         2.0
              Refused    14.0
Marin         18-24       2.0
              25-34      21.0
              35-44       2.0
              45-54      12.0
              55-64      13.0
              65+         3.0
San Mateo     18-24       8.0
              25-34       6.0
              35-44      13.0
              45-54       5.0
              55-64      22.0
              65+        10.0
Santa Clara   25-34       1.0
              35-44       4.0
              45-54       2.0
              55-64       4.0
Solano        25-34      20.0
              35-44      12.5
              45-5

* The above table shows median number of public transport trips to sf undertaken by each outside sf resident belonging to different age/country groups
* Cruise should target cohorts with median number of trips >= 20 